In [33]:
import pandas as pd
import re

ggLink = pd.read_csv('ggLink.csv', sep='\t')
print(ggLink[['gene1','gene2','minResCount']])

         gene1  gene2  minResCount
0            2     C2            3
1            2     C3            8
2            2    C4B            8
3            2  C4BPA            2
4            2     C5            8
...        ...    ...          ...
1322480  ZWINT  ZWINT         3828
1322481   ZXDA   ZXDA            5
1322482   ZXDA   ZXDC            5
1322483   ZXDC   ZXDC            5
1322484    ZYX    ZYX            6

[1322485 rows x 3 columns]


In [76]:
ggLink[ggLink['minResCount']==0]

,gene1,gene2,linkTypes,pairCount,oppCount,docCount,dbList,minResCount,snippet,context
15,A1BG,ERVK-6,text,0,1,1,NaN,0,AHF1 and AHF2 were of the same amino acid sequ...,Hemorrhage (1)
25,A1CF,APOBEC3D,ppi,0,0,0,string,0,NaN,NaN
26,A1CF,APOBEC3F,ppi,0,0,0,string,0,NaN,NaN
27,A1CF,APOBEC3G,ppi,0,0,0,string,0,NaN,NaN
38,A2M,AKT1,text,1,0,1,NaN,0,The transient expression of constitutively act...,NaN
...,...,...,...,...,...,...,...,...,...,...
1322463,ZP1,ZP3,"pwy,text,rev",0,1,1,reactome,0,Neither Zp3 transcripts nor ZP3 protein was de...,"Infertility, Female (1)"
1322464,ZP1,ZP4,"pwy,fwd",0,0,0,reactome,0,NaN,NaN
1322465,ZP2,ZP3,"pwy,text,fwd",0,1,1,reactome,0,Neither Zp3 transcripts nor ZP3 protein was de...,"Infertility, Female (1)"
1322466,ZP2,ZP4,"pwy,fwd",0,0,0,reactome,0,NaN,NaN


In [16]:
def remove_version(x):
    if '.' in x:
        return x.split('.')[0]
    return x

gtf_file_path = '/work/h2020deciderficarra_shared/TCGA/OV/project_n16_data/gencode.v47.annotation.gtf'
gtf = pd.read_csv(gtf_file_path, sep="\t", header=None, comment='#')
gtf.columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']

parameters = ['gene_id', 'gene_type', 'gene_name']
for p in parameters:
    gtf[p] = gtf['attribute'].apply(lambda x: re.findall(rf'{p} "([^"]*)"', x)[0] if rf'{p} "' in x else np.nan)

gtf.drop(['attribute','source','score','strand','frame'], axis=1, inplace=True)

gtf['gene_id'] = gtf['gene_id'].apply(remove_version)

gtf_pc = gtf[gtf['gene_type'] == 'protein_coding']

# Protein coding set
pc_set = set(gtf_pc['gene_id'].to_list())


In [ ]:
gtf_pc_no_dup = gtf_pc.drop_duplicates(subset=['gene_id'])

In [75]:
gene_name1 = gtf_pc_no_dup[gtf_pc_no_dup['gene_id'] == 'ENSG00000188976']['gene_name'].values[0]
gene_name2 = gtf_pc_no_dup[gtf_pc_no_dup['gene_id'] == 'ENSG00000225921']['gene_name'].values[0]

if not ggLink[(ggLink['gene1'] == gene_name1) & (ggLink['gene2'] == gene_name2)].empty:
    print('si')


si


In [73]:
df_gene = gtf_pc_no_dup[gtf_pc_no_dup['gene_id'] == 'ENSG00000225921']['gene_name']
if not df_gene.empty:
    gene_name2 = df_gene.values[0]
else:
    gene_name2 = ''
print(gene_name2)

NOL7


In [5]:
import pandas as pd
import networkx as nx
import numpy as np
from pathlib import Path
import torch
import json
import os
import matplotlib.pyplot as plt
from torch_geometric.utils.convert import to_networkx, from_networkx

PATH_FOLDER_COPY_NUMBER = "/work/h2020deciderficarra_shared/TCGA/OV/project_n16_data/CopyNumber"
PATH_CASE_ID_STRUCTURE = "/homes/dlupo/Progetto_BioInformatics/AI_for_Bioinformatics_Project/Preprocessing/Final/case_id_and_structure.json"
PATH_GENE_ID_PROTEIN_CODING = "/homes/dlupo/Progetto_BioInformatics/AI_for_Bioinformatics_Project/Preprocessing/Final/gene_id_protein_coding.json"

with open(PATH_CASE_ID_STRUCTURE, 'r') as file:
    file_parsed = json.load(file)

copy_number_folder_list = []
os_list = []
for key in file_parsed.keys():
    copy_number_folder_list.append(file_parsed[key]["files"]["copy_number"])
    os_list.append(file_parsed[key]['os'])
print()
gene_id_list = []
with open(PATH_GENE_ID_PROTEIN_CODING) as json_file:
   gene_id_list = json.load(json_file)

/homes/dlupo/.local/lib/python3.9/site-packages/torch/__config__.py:10: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._show_config()


In [ ]:
def remove_version(x):
    if '.' in x:
        return x.split('.')[0]
    return x

list_df_CNV = []
for root, dirs, files in os.walk(PATH_FOLDER_COPY_NUMBER):
    for dir in dirs:
        for root, dirs, files in os.walk(PATH_FOLDER_COPY_NUMBER + "/" + dir):
            for file in files:
                if file in copy_number_folder_list:
                    parsed_file = pd.read_csv(PATH_FOLDER_COPY_NUMBER + "/" + dir + "/" + file, sep='\t')
                    parsed_file['gene_id'] = parsed_file['gene_id'].apply(remove_version)
                    parsed_file = parsed_file[parsed_file['gene_id'].isin(gene_id_list)]
                    list_df_CNV.append(parsed_file)

list_df_CNV_filled = []
for i in range(len(list_df_CNV)):
    list_df_CNV_filled.append(list_df_CNV[i].fillna(0))

In [77]:
edge = [[],[]]
# nodes = len(list_df_CNV_filled[0])
nodes = 300
df = list_df_CNV_filled[0]
# Aggiunta delle connessioni in base alla sovrapposizione
for f_1_index in range(nodes):
    for f_2_index in range(f_1_index + 1, nodes):
        row1 = df.iloc[f_1_index]
        row2 = df.iloc[f_2_index]

        df_gene = gtf_pc_no_dup[gtf_pc_no_dup['gene_id'] == row1['gene_id']]['gene_name']
        if not df_gene.empty:
            gene_name1 = df_gene.values[0]
        else:
            gene_name1 = ''

        df_gene = gtf_pc_no_dup[gtf_pc_no_dup['gene_id'] == row2['gene_id']]['gene_name']
        if not df_gene.empty:
            gene_name2 = df_gene.values[0]
        else:
            gene_name2 = ''

        if not ggLink[(ggLink['gene1'] == gene_name1) & (ggLink['gene2'] == gene_name2)].empty:
            edge[0].append(f_1_index)
            edge[0].append(f_2_index)
            edge[1].append(f_2_index)
            edge[1].append(f_1_index)
            
print(len(edges[0]))


: 